In [1]:
# Set country to ADB country code of country
country = 'GER'

In [2]:
# Import libraries and functions
import wbdata as wb                                       
import pandas as pd
import numpy as np                                     
import datetime as dt                                     
import docx                                               
from docx.shared import Cm                                
from docx.enum.text import WD_ALIGN_PARAGRAPH             
from matplotlib import pyplot as plt 
from matplotlib import ticker
import random
# import Haver
import calendar
from pandasgui import show

In [3]:
# Set the time period
data_date = dt.datetime(dt.date.today().year - 6, 1, 1), dt.datetime(dt.date.today().year - 1, 1, 1)

# Read Excel file of country codes and country names
country_codes = pd.read_excel('all-data.xlsx', sheet_name='data', index_col='adb_code')

# Generate Country Overview table
country_overview = {'SP.POP.TOTL': 'Population (million)',
                    'SP.POP.GROW': 'Population growth (annual % change)',
                    'AG.LND.ARBL.ZS': 'Arable land (%)',
                    'AG.LND.TOTL.K2': 'Land area (sq. km.)'}

df_country_overview = wb.get_dataframe(country_overview, country=country_codes.iso_code[country], data_date=data_date)
df_country_overview_grouped = pd.DataFrame(df_country_overview.transpose().stack()).groupby(level=0)
df_country_overview_right = df_country_overview_grouped.first()
df_country_overview_left = pd.DataFrame(df_country_overview.transpose().stack()).reset_index(level=1)
df_country_overview_joined = df_country_overview_left.join(df_country_overview_right, rsuffix='_r')
df_country_overview = df_country_overview_joined.groupby(level=0).first()
df_country_overview = df_country_overview[['0', 'date']]
df_country_overview = df_country_overview.reindex(['Population (million)', 'Population growth (annual % change)', 'Land area (sq. km.)', 'Arable land (%)'])
df_country_overview.reset_index(inplace=True)
df_country_overview.columns = ['Indicator', 'Value', 'Year']
df_country_overview.iloc[0, 1] = df_country_overview.iloc[0, 1] / 1e6
df_country_overview = df_country_overview.round(1)
df_country_overview.iat[0, 2] = f'[{df_country_overview.iat[0, 2]}]'
df_country_overview.iat[1, 2] = f'[{df_country_overview.iat[1, 2]}]'
df_country_overview.iat[2, 2] = ''
df_country_overview.iat[3, 2] = ''
df_country_overview.iloc[2, 1] = '{:,.0f}'.format(df_country_overview.iloc[2, 1])
# df_country_overview.drop(columns='Year', inplace=True)

In [4]:
# Generate Projections table
projections = {'NY.GDP.MKTP.KD.ZG': 'GDP growth (%)',
               'FP.CPI.TOTL.ZG': 'Inflation (annual average, %)',
               'BN.CAB.XOKA.GD.ZS': 'Current account balance (% of GDP)'}

df_projections = wb.get_dataframe(projections, country=country_codes.iso_code[country], data_date=data_date)
df_projections = df_projections.reset_index().sort_values(by='date').set_index('date').transpose()
df_projections.reset_index(inplace=True)
df_projections.columns = ['Indicator', '2015', '2016', '2017', '2018', '2019', '2020']
df_projections['2021P'], df_projections['2022P'] = np.nan, np.nan
df_projections = df_projections.round(1).fillna('...')
df_projections.drop(columns=['2015'], inplace=True)

In [5]:
# Generate quarterly indicators table
df_quarterly = pd.read_excel(f"{country}-quarterly.xlsx", sheet_name='data', index_col='date')

# Create a dataframe for the quarterly indicators table
df_quarterly.dropna(inplace=True)
df_quarterly = df_quarterly.last('4Q').transpose().round(1)
df_quarterly.columns =[f"Q{df_quarterly.columns[i].quarter} {df_quarterly.columns[i].year}" for i in range(len(df_quarterly.columns))]
df_quarterly.index.names = ['Indicator']
df_quarterly.reset_index(inplace=True)

In [6]:
# Generate Economic Indicators table
economic_indicators = {'NY.GDP.MKTP.CD': 'Total GDP (billion current $)',
                       'NY.GDP.PCAP.CD': 'GDP per capita (current $)',
                       'GC.REV.XGRT.GD.ZS': 'Revenue (% of GDP)',
                       'GC.NLD.TOTL.GD.ZS': 'Fiscal balance (% of GDP)',
                       'FM.LBL.BMNY.ZG': 'Money supply (M2) growth (%)',
                       'SL.UEM.TOTL.NE.ZS': 'Unemployment rate (% of labor force)',
                       'TX.VAL.MRCH.CD.WT': 'Merchandise exports (% of GDP)',
                       'TM.VAL.MRCH.CD.WT': 'Merchandise imports (% of GDP)',
                       'BX.GSR.NFSV.CD': 'Services exports (% of GDP)',
                       'BM.GSR.NFSV.CD': 'Services imports (% of GDP)',
                       'BX.TRF.PWKR.DT.GD.ZS': 'Remittances (% of GDP)',
                       'BX.KLT.DINV.WD.GD.ZS': 'Foreign direct investment (% of GDP)',
                       'DT.DOD.DECT.GN.ZS': 'Total external debt (% of GDP)',
                       'DT.TDS.DECT.EX.ZS': 'Total external debt service (% of exports)', 
                       'FI.RES.TOTL.MO': 'Gross international reserves ( in months of imports)',
                       'PA.NUS.FCRF': 'Average exchange rate (LCY/USD)'}

df_economic = wb.get_dataframe(economic_indicators, country=country_codes.iso_code[country], data_date=data_date)

# Transformations to get desired units
df_economic = df_economic.reset_index().sort_values(by='date').set_index('date')
merchandise_exports_growth = pd.Series(df_economic.iloc[:, 6].pct_change() * 100)
merchandise_imports_growth = pd.Series(df_economic.iloc[:, 7].pct_change() * 100)
df_economic.iloc[:, [0, 6, 7, 8, 9]] = df_economic.iloc[:, [0, 6, 7, 8, 9]] / 1e9
df_economic.iloc[:, 6] = df_economic.iloc[:, 6] / df_economic.iloc[:, 0] * 100
df_economic.iloc[:, 7] = df_economic.iloc[:, 7] / df_economic.iloc[:, 0] * 100
df_economic.iloc[:, 8] = df_economic.iloc[:, 8] / df_economic.iloc[:, 0] * 100
df_economic.iloc[:, 9] = df_economic.iloc[:, 9] / df_economic.iloc[:, 0] * 100

# Add computed series
merchandise_trade_bal_gdp = pd.Series(df_economic.iloc[:, 6] - df_economic.iloc[:, 7])
services_trade_bal_gdp = pd.Series(df_economic.iloc[:, 8] - df_economic.iloc[:, 9])
df_economic.insert(8, 'Merchandise trade balance (% of GDP)', merchandise_trade_bal_gdp)
df_economic.insert(11, 'Balance of services (% of GDP)', services_trade_bal_gdp)
df_economic.insert(14, 'Merchandise exports growth (%)', merchandise_exports_growth)
df_economic.insert(15, 'Merchandise imports growth (%)', merchandise_imports_growth)

# Format series
df_economic = df_economic.round(1)
df_economic.iloc[:, 0] = df_economic.iloc[:, 0].apply(lambda x : "{0:,.0f}".format(x))
df_economic.iloc[:, 1] = df_economic.iloc[:, 1].apply(lambda x : "{0:,.0f}".format(x))
df_economic.fillna('...', inplace=True)
df_economic = df_economic.transpose()
df_economic.index.names = ['Indicator']
df_economic.reset_index(inplace=True)
df_economic.drop(columns=['2015'], inplace=True)

In [7]:
df_prod_structure = pd.read_csv(f'https://stats.oecd.org/SDMX-JSON/data/SNA_TABLE1/{country_codes.iso_code[country]}.B1G_P119+B1GVA+B1GVB_E+B1GVC+B1GVF+B1GVG_I+B1GVJ+B1GVK+B1GVL+B1GVM_N+B1GVO_Q+B1GVR_U.C/all?startTime={dt.date.today().year - 3}&endTime={dt.date.today().year - 1}&contentType=csv')

if len(df_prod_structure[df_prod_structure['Year']==dt.date.today().year - 1]) >= 6:
    df_prod_structure = df_prod_structure[df_prod_structure['Year']==dt.date.today().year - 1]
    df_prod_structure = pd.DataFrame(df_prod_structure.groupby('Transaction').last()['Value'] / df_prod_structure.groupby('Transaction').last()[['Year', 'Value']].at['Gross value added at basic prices, excluding FISIM', 'Value'] * 100)
    df_prod_structure = df_prod_structure.reindex(['Agriculture, forestry and fishing (ISIC rev4)',
                                                   'Industry, including energy (ISIC rev4)',
                                                   'of which: Manufacturing (ISIC rev4)', 
                                                   'Construction (ISIC rev4)',
                                                   'Services',
                                                   'Distributive trade, repairs; transport; accommod., food serv. (ISIC rev4)',
                                                   'Information and communication (ISIC rev4)',
                                                   'Other branches of services',
                                                   'Financial and insurance activities (ISIC rev4)',
                                                   'Prof., scientific, techn.; admin., support serv. activities (ISIC rev4)',
                                                   'Public admin.; compulsory s.s.; education; human health (ISIC rev4)',
                                                   'Real estate activities (ISIC rev4)',
                                                   'Other service activities (ISIC rev4)',])
    df_prod_structure.at['Industry, including energy (ISIC rev4)', 'Value'] = df_prod_structure.at['Industry, including energy (ISIC rev4)', 'Value'] + df_prod_structure.at['Construction (ISIC rev4)', 'Value']
    df_prod_structure.at['Services', 'Value'] = df_prod_structure.loc['Distributive trade, repairs; transport; accommod., food serv. (ISIC rev4)':].sum()[0]
    df_prod_structure.at['Other branches of services', 'Value'] = df_prod_structure.loc['Financial and insurance activities (ISIC rev4)':].sum()[0]
    df_prod_structure = df_prod_structure.loc[:'Other branches of services']
    df_prod_structure = df_prod_structure.round(1)
    df_prod_structure.index = ['Agriculture', 'Industry', 'Manufacturing', 'Construction', 'Services', 'Trade, accommodation & food services', 'Transportation & storage, information & communication', 'Other branches of services']
    df_prod_structure.index.names = [f'Production Structure, {dt.date.today().year - 1}']
    df_prod_structure.columns = ['% of GDP']
    df_prod_structure.fillna('...', inplace=True)

elif len(df_prod_structure[df_prod_structure['Year']==dt.date.today().year - 2]) >= 6:
    df_prod_structure = df_prod_structure[df_prod_structure['Year']==dt.date.today().year - 2]
    df_prod_structure = pd.DataFrame(df_prod_structure.groupby('Transaction').last()['Value'] / df_prod_structure.groupby('Transaction').last()[['Year', 'Value']].at['Gross value added at basic prices, excluding FISIM', 'Value'] * 100)
    df_prod_structure = df_prod_structure.reindex(['Agriculture, forestry and fishing (ISIC rev4)',
                                                   'Industry, including energy (ISIC rev4)',
                                                   'of which: Manufacturing (ISIC rev4)', 
                                                   'Construction (ISIC rev4)',
                                                   'Services',
                                                   'Distributive trade, repairs; transport; accommod., food serv. (ISIC rev4)',
                                                   'Information and communication (ISIC rev4)',
                                                   'Other branches of services',
                                                   'Financial and insurance activities (ISIC rev4)',
                                                   'Prof., scientific, techn.; admin., support serv. activities (ISIC rev4)',
                                                   'Public admin.; compulsory s.s.; education; human health (ISIC rev4)',
                                                   'Real estate activities (ISIC rev4)',
                                                   'Other service activities (ISIC rev4)',])
    df_prod_structure.at['Industry, including energy (ISIC rev4)', 'Value'] = df_prod_structure.at['Industry, including energy (ISIC rev4)', 'Value'] + df_prod_structure.at['Construction (ISIC rev4)', 'Value']
    df_prod_structure.at['Services', 'Value'] = df_prod_structure.loc['Distributive trade, repairs; transport; accommod., food serv. (ISIC rev4)':].sum()[0]
    df_prod_structure.at['Other branches of services', 'Value'] = df_prod_structure.loc['Financial and insurance activities (ISIC rev4)':].sum()[0]
    df_prod_structure = df_prod_structure.loc[:'Other branches of services']
    df_prod_structure = df_prod_structure.round(1)
    df_prod_structure.index = ['Agriculture', 'Industry', 'Manufacturing', 'Construction', 'Services', 'Trade, accommodation & food services', 'Transportation & storage, information & communication', 'Other branches of services']
    df_prod_structure.index.names = [f'Production Structure, {dt.date.today().year - 2}']
    df_prod_structure.columns = ['% of GDP']
    df_prod_structure.fillna('...', inplace=True)

elif len(df_prod_structure[df_prod_structure['Year']==dt.date.today().year - 3]) >= 6:
    df_prod_structure = df_prod_structure[df_prod_structure['Year']==dt.date.today().year - 3]
    df_prod_structure = pd.DataFrame(df_prod_structure.groupby('Transaction').last()['Value'] / df_prod_structure.groupby('Transaction').last()[['Year', 'Value']].at['Gross value added at basic prices, excluding FISIM', 'Value'] * 100)
    df_prod_structure = df_prod_structure.reindex(['Agriculture, forestry and fishing (ISIC rev4)',
                                                   'Industry, including energy (ISIC rev4)',
                                                   'of which: Manufacturing (ISIC rev4)', 
                                                   'Construction (ISIC rev4)',
                                                   'Services',
                                                   'Distributive trade, repairs; transport; accommod., food serv. (ISIC rev4)',
                                                   'Information and communication (ISIC rev4)',
                                                   'Other branches of services',
                                                   'Financial and insurance activities (ISIC rev4)',
                                                   'Prof., scientific, techn.; admin., support serv. activities (ISIC rev4)',
                                                   'Public admin.; compulsory s.s.; education; human health (ISIC rev4)',
                                                   'Real estate activities (ISIC rev4)',
                                                   'Other service activities (ISIC rev4)',])
    df_prod_structure.at['Industry, including energy (ISIC rev4)', 'Value'] = df_prod_structure.at['Industry, including energy (ISIC rev4)', 'Value'] + df_prod_structure.at['Construction (ISIC rev4)', 'Value']
    df_prod_structure.at['Services', 'Value'] = df_prod_structure.loc['Distributive trade, repairs; transport; accommod., food serv. (ISIC rev4)':].sum()[0]
    df_prod_structure.at['Other branches of services', 'Value'] = df_prod_structure.loc['Financial and insurance activities (ISIC rev4)':].sum()[0]
    df_prod_structure = df_prod_structure.loc[:'Other branches of services']
    df_prod_structure = df_prod_structure.round(1)
    df_prod_structure.index = ['Agriculture', 'Industry', 'Manufacturing', 'Construction', 'Services', 'Trade, accommodation & food services', 'Transportation & storage, information & communication', 'Other branches of services']
    df_prod_structure.index.names = [f'Production Structure, {dt.date.today().year - 3}']
    df_prod_structure.columns = ['% of GDP']
    df_prod_structure.fillna('...', inplace=True)

In [8]:
df_empl_structure = pd.read_csv(f'https://stats.oecd.org/SDMX-JSON/data/SNA_TABLE3/{country_codes.iso_code[country]}.ETO+ETOVA+ETOVB_E+ETOVC+ETOVF+ETOVG_I+ETOVJ+ETOVK+ETOVL+ETOVM_N+ETOVO_Q+ETOVR_U.PER/all?startTime={dt.date.today().year - 3}&endTime={dt.date.today().year - 1}&contentType=csv')

if len(df_empl_structure[df_empl_structure['Year']==dt.date.today().year - 1]) >= 6:
    df_empl_structure = df_empl_structure[df_empl_structure['Year']==dt.date.today().year - 1]
    df_empl_structure = pd.DataFrame(df_empl_structure.groupby('Transaction').last()['Value'] / df_empl_structure.groupby('Transaction').last()[['Year', 'Value']].at['Total employment, domestic concept', 'Value'] * 100)
    df_empl_structure = df_empl_structure.reindex(['Agriculture, forestry and fishing (ISIC rev4)',
                                                   'Industry, including energy (ISIC rev4)',
                                                   'of which: Manufacturing (ISIC rev4)', 
                                                   'Construction (ISIC rev4)',
                                                   'Services',
                                                   'Distributive trade, repairs; transport; accommod., food serv. (ISIC rev4)',
                                                   'Information and communication (ISIC rev4)',
                                                   'Other branches of services',
                                                   'Financial and insurance activities (ISIC rev4)',
                                                   'Prof., scientific, techn.; admin., support serv. activities (ISIC rev4)',
                                                   'Public admin.; compulsory s.s.; education; human health (ISIC rev4)',
                                                   'Real estate activities (ISIC rev4)',
                                                   'Other service activities (ISIC rev4)',])
    df_empl_structure.at['Industry, including energy (ISIC rev4)', 'Value'] = df_empl_structure.at['Industry, including energy (ISIC rev4)', 'Value'] + df_empl_structure.at['Construction (ISIC rev4)', 'Value']
    df_empl_structure.at['Services', 'Value'] = df_empl_structure.loc['Distributive trade, repairs; transport; accommod., food serv. (ISIC rev4)':].sum()[0]
    df_empl_structure.at['Other branches of services', 'Value'] = df_empl_structure.loc['Financial and insurance activities (ISIC rev4)':].sum()[0]
    df_empl_structure = df_empl_structure.loc[:'Other branches of services']
    df_empl_structure = df_empl_structure.round(1)
    df_empl_structure.index = ['Agriculture', 'Industry', 'Manufacturing', 'Construction', 'Services', 'Trade, accommodation & food services', 'Transportation & storage, information & communication', 'Other branches of services']
    df_empl_structure.index.names = [f'Employment Share, {dt.date.today().year - 1}']
    df_empl_structure.columns = [f'Employment Share, {dt.date.today().year - 2}']
    df_empl_structure.fillna('...', inplace=True)

elif len(df_empl_structure[df_empl_structure['Year']==dt.date.today().year - 2]) >= 6:
    df_empl_structure = df_empl_structure[df_empl_structure['Year']==dt.date.today().year - 2]
    df_empl_structure = pd.DataFrame(df_empl_structure.groupby('Transaction').last()['Value'] / df_empl_structure.groupby('Transaction').last()[['Year', 'Value']].at['Total employment, domestic concept', 'Value'] * 100)
    df_empl_structure = df_empl_structure.reindex(['Agriculture, forestry and fishing (ISIC rev4)',
                                                   'Industry, including energy (ISIC rev4)',
                                                   'of which: Manufacturing (ISIC rev4)', 
                                                   'Construction (ISIC rev4)',
                                                   'Services',
                                                   'Distributive trade, repairs; transport; accommod., food serv. (ISIC rev4)',
                                                   'Information and communication (ISIC rev4)',
                                                   'Other branches of services',
                                                   'Financial and insurance activities (ISIC rev4)',
                                                   'Prof., scientific, techn.; admin., support serv. activities (ISIC rev4)',
                                                   'Public admin.; compulsory s.s.; education; human health (ISIC rev4)',
                                                   'Real estate activities (ISIC rev4)',
                                                   'Other service activities (ISIC rev4)',])
    df_empl_structure.at['Industry, including energy (ISIC rev4)', 'Value'] = df_empl_structure.at['Industry, including energy (ISIC rev4)', 'Value'] + df_empl_structure.at['Construction (ISIC rev4)', 'Value']
    df_empl_structure.at['Services', 'Value'] = df_empl_structure.loc['Distributive trade, repairs; transport; accommod., food serv. (ISIC rev4)':].sum()[0]
    df_empl_structure.at['Other branches of services', 'Value'] = df_empl_structure.loc['Financial and insurance activities (ISIC rev4)':].sum()[0]
    df_empl_structure = df_empl_structure.loc[:'Other branches of services']
    df_empl_structure = df_empl_structure.round(1)
    df_empl_structure.index = ['Agriculture', 'Industry', 'Manufacturing', 'Construction', 'Services', 'Trade, accommodation & food services', 'Transportation & storage, information & communication', 'Other branches of services']
    df_empl_structure.index.names = [f'Employment Share, {dt.date.today().year - 2}']
    df_empl_structure.columns = [f'Employment Share, {dt.date.today().year - 2}']
    df_empl_structure.fillna('...', inplace=True)

elif len(df_empl_structure[df_empl_structure['Year']==dt.date.today().year - 3]) >= 6:
    df_empl_structure = df_empl_structure[df_empl_structure['Year']==dt.date.today().year - 3]
    df_empl_structure = pd.DataFrame(df_empl_structure.groupby('Transaction').last()['Value'] / df_empl_structure.groupby('Transaction').last()[['Year', 'Value']].at['Total employment, domestic concept', 'Value'] * 100)
    df_empl_structure = df_empl_structure.reindex(['Agriculture, forestry and fishing (ISIC rev4)',
                                                   'Industry, including energy (ISIC rev4)',
                                                   'of which: Manufacturing (ISIC rev4)', 
                                                   'Construction (ISIC rev4)',
                                                   'Services',
                                                   'Distributive trade, repairs; transport; accommod., food serv. (ISIC rev4)',
                                                   'Information and communication (ISIC rev4)',
                                                   'Other branches of services',
                                                   'Financial and insurance activities (ISIC rev4)',
                                                   'Prof., scientific, techn.; admin., support serv. activities (ISIC rev4)',
                                                   'Public admin.; compulsory s.s.; education; human health (ISIC rev4)',
                                                   'Real estate activities (ISIC rev4)',
                                                   'Other service activities (ISIC rev4)',])
    df_empl_structure.at['Industry, including energy (ISIC rev4)', 'Value'] = df_empl_structure.at['Industry, including energy (ISIC rev4)', 'Value'] + df_empl_structure.at['Construction (ISIC rev4)', 'Value']
    df_empl_structure.at['Services', 'Value'] = df_empl_structure.loc['Distributive trade, repairs; transport; accommod., food serv. (ISIC rev4)':].sum()[0]
    df_empl_structure.at['Other branches of services', 'Value'] = df_empl_structure.loc['Financial and insurance activities (ISIC rev4)':].sum()[0]
    df_empl_structure = df_empl_structure.loc[:'Other branches of services']
    df_empl_structure = df_empl_structure.round(1)
    df_empl_structure.index = ['Agriculture', 'Industry', 'Manufacturing', 'Construction', 'Services', 'Trade, accommodation & food services', 'Transportation & storage, information & communication', 'Other branches of services']
    df_empl_structure.index.names = [f'Employment Share, {dt.date.today().year - 3}']
    df_empl_structure.columns = [f'Employment Share, {dt.date.today().year - 2}']
    df_empl_structure.fillna('...', inplace=True)

In [9]:
df_expe_structure = pd.read_csv(f'https://stats.oecd.org/SDMX-JSON/data/SNA_TABLE1/{country_codes.iso_code[country]}.B1_GE+P31S14_S15+P3S13+P5+B11+P6+P7+DB1_GE.C/all?startTime={dt.date.today().year - 3}&endTime={dt.date.today().year - 1}&contentType=csv')

if len(df_expe_structure[df_expe_structure['Year']==dt.date.today().year - 1]) >= 6:
    df_expe_structure = df_expe_structure[df_expe_structure['Year']==dt.date.today().year - 1]
    df_expe_structure = pd.DataFrame(df_expe_structure.groupby('Transaction').last()['Value'] / df_expe_structure.groupby('Transaction').last()[['Year', 'Value']].at['Gross domestic product (expenditure approach)', 'Value'] * 100)
    df_expe_structure = df_expe_structure.reindex(['Households and Non-profit institutions serving households',
                                               'Final consumption expenditure of general government',
                                               'Gross capital formation',
                                               'Public investment',
                                               'Private investment',
                                               'External balance of goods and services',
                                               'Exports of goods and services',
                                               'Imports of goods and services',
                                               'Statistical discrepancy'])
    df_expe_structure.index = ['Private consumption', 'Government consumption', 'Investment', 'Public', 'Private', 'Net exports', 'Exports', 'Imports', 'Statistical discrepancy']
    df_expe_structure = df_expe_structure.round(1)
    df_expe_structure.index.names = [f'GDP by Expenditure, {dt.date.today().year - 1}']
    df_expe_structure.columns = ['% of GDP']
    df_expe_structure.fillna('...', inplace=True)
    df_expe_structure.reset_index(inplace=True)

elif len(df_expe_structure[df_expe_structure['Year']==dt.date.today().year - 2]) >= 6:
    df_expe_structure = df_expe_structure[df_expe_structure['Year']==dt.date.today().year - 2]
    df_expe_structure = pd.DataFrame(df_expe_structure.groupby('Transaction').last()['Value'] / df_expe_structure.groupby('Transaction').last()[['Year', 'Value']].at['Gross domestic product (expenditure approach)', 'Value'] * 100)
    df_expe_structure = df_expe_structure.reindex(['Households and Non-profit institutions serving households',
                                               'Final consumption expenditure of general government',
                                               'Gross capital formation',
                                               'Public investment',
                                               'Private investment',
                                               'External balance of goods and services',
                                               'Exports of goods and services',
                                               'Imports of goods and services',
                                               'Statistical discrepancy'])
    df_expe_structure.index = ['Private consumption', 'Government consumption', 'Investment', 'Public', 'Private', 'Net exports', 'Exports', 'Imports', 'Statistical discrepancy']
    df_expe_structure = df_expe_structure.round(1)
    df_expe_structure.index.names = [f'GDP by Expenditure, {dt.date.today().year - 2}']
    df_expe_structure.columns = ['% of GDP']
    df_expe_structure.fillna('...', inplace=True)
    df_expe_structure.reset_index(inplace=True)

elif len(df_expe_structure[df_expe_structure['Year']==dt.date.today().year - 3]) >= 6:
    df_expe_structure = df_expe_structure[df_expe_structure['Year']==dt.date.today().year - 3]
    df_expe_structure = pd.DataFrame(df_expe_structure.groupby('Transaction').last()['Value'] / df_expe_structure.groupby('Transaction').last()[['Year', 'Value']].at['Gross domestic product (expenditure approach)', 'Value'] * 100)
    df_expe_structure = df_expe_structure.reindex(['Households and Non-profit institutions serving households',
                                               'Final consumption expenditure of general government',
                                               'Gross capital formation',
                                               'Public investment',
                                               'Private investment',
                                               'External balance of goods and services',
                                               'Exports of goods and services',
                                               'Imports of goods and services',
                                               'Statistical discrepancy'])
    df_expe_structure.index = ['Private consumption', 'Government consumption', 'Investment', 'Public', 'Private', 'Net exports', 'Exports', 'Imports', 'Statistical discrepancy']
    df_expe_structure = df_expe_structure.round(1)
    df_expe_structure.index.names = [f'GDP by Expenditure, {dt.date.today().year - 3}']
    df_expe_structure.columns = ['% of GDP']
    df_expe_structure.fillna('...', inplace=True)
    df_expe_structure.reset_index(inplace=True)

df_structure = df_prod_structure.join(df_empl_structure).reset_index().join(df_expe_structure, rsuffix='_r', how='right').fillna('')
df_structure.columns = ['Production Structure, 2020', '% of GDP', 'Employment Share, 2020', 'GDP by Expenditure, 2020', '% of GDP']

In [10]:
uitoken = 'b3f9aeb66ba24558248a2c38acebc1f'

# https://comtrade.un.org/api/get/plus?max=100000&type=C&freq=A&px=HS&ps={dt.date.today().year - 2}&r={country_codes.comtrade_code[country]}&p=all&rg=1&cc=TOTAL&uitoken={uitoken}&fmt=csv

url_destination = f"https://comtrade.un.org/api/get?max=100000&type=C&freq=A&px=HS&ps={dt.date.today().year - 1}&r={country_codes.comtrade_code[country]}&p=all&rg=2&cc=TOTAL&uitoken={uitoken}&fmt=csv"
df_destination = pd.read_csv(url_destination)
# mask1 = (df_destination['Partner']=='World') & (df_destination['Mode of Transport']=='All MOTs') & (df_destination['2nd Partner']=='World')
mask1 = df_destination['Partner']=='World'
total_exports = df_destination[mask1]['Trade Value (US$)'].values[0]
# mask2 = (~(df_destination['Partner'] == 'World')) & (df_destination['Mode of Transport'] == 'All MOTs') & (df_destination['2nd Partner']=='World')
mask2 = ~(df_destination['Partner'] == 'World')
df_destination = df_destination[mask2]
df_destination = df_destination.assign(share=lambda x: (x['Trade Value (US$)'] / total_exports * 100).round(1))
top_destination = df_destination.sort_values(by='share', ascending=False)[['Partner', 'share']].reset_index(drop=True).head(5)
top_destination.columns = [f'Main Destinations of Exports, {dt.date.today().year - 1}', '% of Total']

# Download import data
url_origin = f"https://comtrade.un.org/api/get?max=100000&type=C&freq=A&px=HS&ps={dt.date.today().year - 1}&r={country_codes.comtrade_code[country]}&p=all&rg=1&cc=TOTAL&uitoken={uitoken}&fmt=csv"
df_origin = pd.read_csv(url_origin)
# mask1 = (df_origin['Partner']=='World') & (df_origin['Mode of Transport']=='All MOTs') & (df_origin['2nd Partner']=='World')
mask1 = df_origin['Partner']=='World'
total_imports = df_origin[mask1]['Trade Value (US$)'].values[0]
# mask2 = (~(df_origin['Partner'] == 'World')) & (df_origin['Mode of Transport'] == 'All MOTs') & (df_origin['2nd Partner']=='World')
mask2 = ~(df_origin['Partner'] == 'World')
df_origin = df_origin[mask2]
df_origin = df_origin.assign(share=lambda x: (x['Trade Value (US$)'] / total_imports * 100).round(1))
top_origin = df_origin.sort_values(by='share', ascending=False)[['Partner', 'share']].reset_index(drop=True).head(5)
top_origin.columns = [f'Main Sources of Imports, {dt.date.today().year - 1}', '% of Total']


url_exports = f"https://comtrade.un.org/api/get?max=100000&type=C&freq=A&px=HS&ps={dt.date.today().year - 1}&r={country_codes.comtrade_code[country]}&p=0&rg=2&cc=AG2&uitoken={uitoken}&fmt=csv"
df_exports = pd.read_csv(url_exports)
# mask = (df_exports['Mode of Transport']=='All MOTs') & (df_exports['2nd Partner']=='World')
# mask = df_exports['Mode of Transport']=='All MOTs'
# df_exports = df_exports[mask]
df_exports = df_exports.assign(share=lambda x: (x['Trade Value (US$)'] / total_exports * 100).round(1))
top_exports = df_exports.sort_values(by='share', ascending=False)[['Commodity', 'share']].reset_index(drop=True).head(5)
top_exports.columns = [f'Principal Exports, {dt.date.today().year - 1}', '% of Total']


url_imports = f"https://comtrade.un.org/api/get?max=100000&type=C&freq=A&px=HS&ps={dt.date.today().year - 1}&r={country_codes.comtrade_code[country]}&p=0&rg=1&cc=AG2&uitoken={uitoken}&fmt=csv"
df_imports = pd.read_csv(url_imports)
# mask = (df_imports['Mode of Transport']=='All MOTs') & (df_imports['2nd Partner']=='World')
# mask = df_imports['Mode of Transport']=='All MOTs'
# df_imports = df_imports[mask]
df_imports = df_imports.assign(share=lambda x: (x['Trade Value (US$)'] / total_imports * 100).round(1))
top_imports = df_imports.sort_values(by='share', ascending=False)[['Commodity', 'share']].reset_index(drop=True).head(5)
top_imports.columns = [f'Principal Imports, {dt.date.today().year - 1}', '% of Total']

df_trade_partners = pd.concat([top_destination, top_origin], axis=1)
df_trade_commodities = pd.concat([top_exports, top_imports], axis=1)


In [11]:
# Generate Poverty Indicators table
poverty_indicators = {'SI.POV.DDAY': 'Population living less than $1.9 a day (%)',
                      'SI.POV.NAHC': 'Population below national poverty line (%)',
                      'SH.STA.MALN.ZS': 'Underweight children under 5 years old (%)',
                      'SE.PRM.NENR': 'Net enrollment ratio in primary education, Total (%)',
                      'SE.PRM.NENR.FE': 'Net enrollment ratio in primary education, Female (%)',
                      'SE.PRM.NENR.MA': 'Net enrollment ratio in primary education, Male (%)',
                      'SE.ADT.LITR.ZS': 'Adult literacy rate (%)',
                    #   'SH.STA.MMRT': 'Maternal mortality ratio (modeled estimate, per 100,000 live births)',
                      'SH.STA.MMRT.NE': 'Maternal mortality ratio (national estimate, per 100,000 live births)',
                      'SP.DYN.IMRT.IN': 'Infant mortality rate (below 1 year/per 1,000 live births)',
                      'SP.DYN.LE00.IN': 'Life expectancy at birth (years)',
                      'EN.ATM.CO2E.PC': 'CO2 emissions (metric tons per capita)',
                      'SH.H2O.BASW.ZS': 'Population with access to safe water (%)',
                      'SH.STA.BASS.ZS': 'Population with access to basic sanitation (%)'}
df_poverty = wb.get_dataframe(poverty_indicators, country=[f'{country_codes.iso_code[country]}', 'OED'])

df_poverty_l = pd.DataFrame(df_poverty.stack(), columns=['value'])
df_poverty_l = df_poverty_l.reset_index().groupby(['country', 'level_2']).first()
df_poverty_l.reset_index(inplace=True)
df_poverty_l['date'] = pd.to_numeric(df_poverty_l['date'])
df_poverty_l = df_poverty_l.pivot_table(index='level_2', columns='country', values='value')
df_poverty_l.reset_index(inplace=True)

df_poverty_r = pd.DataFrame(df_poverty.stack(), columns=['value'])
df_poverty_r = df_poverty_r.reset_index().groupby(['country', 'level_2']).first()
df_poverty_r.reset_index(inplace=True)
df_poverty_r['date'] = pd.to_numeric(df_poverty_r['date'])
df_poverty_r = df_poverty_r.pivot_table(index='level_2', columns='country', values='date')
df_poverty_r.reset_index(inplace=True)

df_poverty = df_poverty_l.join(df_poverty_r, rsuffix=' Year')
df_poverty = df_poverty.iloc[:, [0, 1, 4, 2, 5]]
df_poverty.columns = ['Indicator', f'{country}', 'Year', 'OECD', 'Year']
df_poverty = df_poverty.round(1)
df_poverty.iloc[:, 2] = df_poverty.iloc[:, 2].apply(lambda x : "{:.0f}".format(x))
df_poverty.iloc[:, 4] = df_poverty.iloc[:, 4].apply(lambda x : "{:.0f}".format(x))


df_poverty = df_poverty.set_index('Indicator').reindex(['Population living less than $1.9 a day (%)',
                                                        'Population below national poverty line (%)',
                                                        'Underweight children under 5 years old (%)',
                                                        'Net enrollment ratio in primary education, Total (%)',
                                                        'Net enrollment ratio in primary education, Female (%)',
                                                        'Net enrollment ratio in primary education, Male (%)',
                                                        'Adult literacy rate (%)',
                                                        # 'Maternal mortality ratio (modeled estimate, per 100,000 live births)',
                                                        'Maternal mortality ratio (national estimate, per 100,000 live births)',
                                                        'Infant mortality rate (below 1 year/per 1,000 live births)',
                                                        'Life expectancy at birth (years)',
                                                        'CO2 emissions (metric tons per capita)',
                                                        'Population with access to safe water (%)',
                                                        'Population with access to basic sanitation (%)'])
df_poverty.reset_index(inplace=True)
df_poverty = df_poverty.fillna('...')

In [26]:
df_poverty_l.join(df_poverty_r, rsuffix=' Year').iloc[:, [0, 1, 4, 2, 5]]

country,level_2,Germany,Germany Year,OECD members,OECD members Year
0,CO2 emissions (metric tons per capita),8.840129,2016.0,8.980592,2016.0
1,"Infant mortality rate (below 1 year/per 1,000 ...",3.200000,2019.0,5.942955,2019.0
2,Life expectancy at birth (years),80.892683,2018.0,80.077530,2018.0
3,"Maternal mortality ratio (national estimate, p...",3.000000,2015.0,NaN,NaN
4,"Net enrollment ratio in primary education, Fem...",91.322720,2016.0,95.539720,2018.0
5,"Net enrollment ratio in primary education, Mal...",89.562590,2016.0,95.649220,2018.0
6,"Net enrollment ratio in primary education, Tot...",90.142860,2017.0,95.595800,2018.0
7,Population below national poverty line (%),14.800000,2018.0,NaN,NaN
8,Population living less than $1.9 a day (%),0.000000,2016.0,NaN,NaN
9,Population with access to basic sanitation (%),99.225038,2017.0,98.201634,2017.0


In [12]:
# Create a blank Word document where the output will be exported
doc = docx.Document()
doc.add_heading(f'{country}: Country Information Notes')

In [13]:
# Export Country Overview table to Word
doc.add_heading('Country Overview', level=1)
table1 = doc.add_table(df_country_overview.shape[0]+1, df_country_overview.shape[1])
for j in range(df_country_overview.shape[-1]):
    table1.cell(0, j).text = df_country_overview.columns[j]
for i in range(df_country_overview.shape[0]):
    for j in range(df_country_overview.shape[-1]):
        table1.cell(i+1,j).text = str(df_country_overview.values[i,j])
table1.style = 'Table Grid'

doc.add_heading('Economic and Social Indicators', level=1)

#Export the Projections table
doc.add_heading('Selected Economic Indicators (2015-2019) and Projections (2020-2021)', level=2)
table6 = doc.add_table(df_projections.shape[0]+1, df_projections.shape[1])
for j in range(df_projections.shape[-1]):
    table6.cell(0, j).text = df_projections.columns[j]
for i in range(df_projections.shape[0]):
    for j in range(df_projections.shape[-1]):
        table6.cell(i+1,j).text = str(df_projections.values[i,j])
table6.style = 'Table Grid'
table6.allow_autofit = True

#Export the Projections table
doc.add_heading('Selected Economic Indicators (4 most recent quarters)', level=2)
table7 = doc.add_table(df_quarterly.shape[0]+1, df_quarterly.shape[1])
for j in range(df_quarterly.shape[-1]):
    table7.cell(0, j).text = df_quarterly.columns[j]
for i in range(df_quarterly.shape[0]):
    for j in range(df_quarterly.shape[-1]):
        table7.cell(i+1,j).text = str(df_quarterly.values[i,j])
table7.style = 'Table Grid'

doc.add_heading('Key Economic Indicators', level=2)
table2 = doc.add_table(df_economic.shape[0]+1, df_economic.shape[1])
for j in range(df_economic.shape[-1]):
    table2.cell(0, j).text = df_economic.columns[j]
for i in range(df_economic.shape[0]):
    for j in range(df_economic.shape[-1]):
        table2.cell(i+1,j).text = str(df_economic.values[i,j])
table2.style = 'Table Grid'

# Export the Economic Structure table to Word
doc.add_heading('Economic Structure', level=2)
table3 = doc.add_table(df_structure.shape[0]+1, df_structure.shape[1])
for j in range(df_structure.shape[-1]):
    table3.cell(0, j).text = df_structure.columns[j]
for i in range(df_structure.shape[0]):
    for j in range(df_structure.shape[-1]):
        table3.cell(i+1,j).text = str(df_structure.values[i,j])
table3.style = 'Table Grid'

# Export the Trade Country table to Word
doc.add_heading('Trade Structure: Exports Destinations and Import Origins', level=2)
table4 = doc.add_table(df_trade_partners.shape[0]+1, df_trade_partners.shape[1])
for j in range(df_trade_partners.shape[-1]):
    table4.cell(0, j).text = df_trade_partners.columns[j]
for i in range(df_trade_partners.shape[0]):
    for j in range(df_trade_partners.shape[-1]):
        table4.cell(i+1,j).text = str(df_trade_partners.values[i,j])
table4.style = 'Table Grid'

# Export the Trade Items table to Word
doc.add_heading('Trade Structure: Exports and Imports Products', level=2)
table5 = doc.add_table(df_trade_commodities.shape[0]+1, df_trade_commodities.shape[1])
for j in range(df_trade_commodities.shape[-1]):
    table5.cell(0, j).text = df_trade_commodities.columns[j]
for i in range(df_trade_commodities.shape[0]):
    for j in range(df_trade_commodities.shape[-1]):
        table5.cell(i+1,j).text = str(df_trade_commodities.values[i,j])
table5.style = 'Table Grid'



#Export the Poverty Indicators table
doc.add_heading('Key Poverty and Social Indicators', level=2)
table7 = doc.add_table(df_poverty.shape[0]+1, df_poverty.shape[1])
for j in range(df_poverty.shape[-1]):
    table7.cell(0, j).text = df_poverty.columns[j]
for i in range(df_poverty.shape[0]):
    for j in range(df_poverty.shape[-1]):
        table7.cell(i+1,j).text = str(df_poverty.values[i,j])
table7.style = 'Table Grid'

doc.save(f'./{country}_CIN.docx')